# SUBMISSION NOTEBOOK

## SETUP

#### imports 

In [1]:
import pandas as pd
import os
import datetime
import subprocess
import re
from omegaconf import OmegaConf
import pathlib
from tqdm.auto import tqdm
import itertools

In [2]:
import wandb

#### CONFIG:

In [3]:
CODE_BASE = '/sc-projects/sc-proj-ukb-cvd/code/RetinalRisk'
TAG = '230426'
PARTITIONS = [4,5,7,9,10,20]
IMG_SHAPE = [1,3,384,384]

#### get wandb runs

In [4]:
api = wandb.Api()
entity, project = "cardiors", "retina"
runs = api.runs(entity + "/" + project) 

In [5]:
runs[0].config['best_checkpoint']

'/sc-projects/sc-proj-ukb-cvd/results/models/retina/333mvddg/checkpoints/epoch=92-step=36828.ckpt'

In [6]:
run_list = []
for run in tqdm(runs): 
    if TAG in run.tags:
        run_list.append(
            {
                "id": run.id, #path[-1]
                "name": run.name,
                "tags": run.tags,
                "partition": eval(run.config['_content']['datamodule'])['partition'],
                "best_ckpt": run.config['best_checkpoint'] if 'best_checkpoint' in run.config.keys() else None
            }
        )

  0%|          | 0/292 [00:00<?, ?it/s]

In [7]:
runs_df = pd.DataFrame(run_list)

In [8]:
runs_df = runs_df[runs_df.partition.isin(PARTITIONS)]
runs_df

,id,name,tags,partition,best_ckpt
0,3n2one7r,230426_fullrun_retina,"[230426, baseline_data, image]",20,/sc-projects/sc-proj-ukb-cvd/results/models/re...
1,1xcmy2ys,230426_fullrun_retina,"[230426, baseline_data, image]",10,/sc-projects/sc-proj-ukb-cvd/results/models/re...
2,37ps9kby,230426_fullrun_retina,"[230426, baseline_data, image]",9,/sc-projects/sc-proj-ukb-cvd/results/models/re...
3,lzl9g0gn,230426_fullrun_retina,"[230426, baseline_data, image]",7,/sc-projects/sc-proj-ukb-cvd/results/models/re...
4,3uyg9080,230426_fullrun_retina,"[230426, baseline_data, image]",4,/sc-projects/sc-proj-ukb-cvd/results/models/re...
6,16f0zmet,230426_fullrun_retina,"[230426, baseline_data, image]",5,/sc-projects/sc-proj-ukb-cvd/results/models/re...


In [9]:
yaml = """
setup:
  entity: cardiors
  project: retina
  group: null
  name: onnx_export
  root:
    charite-hpc: /sc-projects/sc-proj-ukb-cvd
    eils-hpc: /data/analysis/ag-reils/ag-reils-shared/cardioRS
  data_path: data/2_datasets_pre/211110_anewbeginning/artifacts
  output_path: results/models
  use_data_artifact_if_available: false
  data: null
  restore_id: null
  data_identifier: WandBBaselineData:latest
  tags:
  - baseline_data
head:
  model_type: MLP
  dropout: 0.5
  kwargs:
    num_hidden: 512
    num_layers: 2
    detach_clf: false
    initial_dropout: 0.0
datamodule:
  covariates: []
  augmentation:
    train:
      RandomRotation:
        degrees: 25
      CenterCrop:
        size: 384
      RandomHorizontalFlip:
        p: 0.5
      RandomVerticalFlip:
        p: 0.5
      RandomGrayscale:
        p: 0.1
      RandomPosterize:
        bits: 4
        p: 0.2
      RandomAdjustSharpness:
        sharpness_factor: 2
        p: 0.3
      RandomAutocontrast:
        p: 0.3
    valid:
      CenterCrop:
        size: 384
    test:
      CenterCrop:
        size: 384
  batch_size: 256
  partition: 0
  num_workers: 16
  img_root: /sc-projects/sc-proj-ukb-cvd/data/retina/preprocessed/preprocessed
  img_visit: 0
  img_size_to_gpu: 420
  img_crop_ratio:
   train:
    - 0.3
    - 0.4
    - 0.5
    - 0.6
    - 0.7
    - 0.8
   test: 0.5
   valid: 0.8
  img_testtime_crop_ratio:
   - 0.66
  img_n_testtime_views: 50
  label_definition:
    all_cause_death: true
    phecodes: true
    custom: /sc-projects/sc-proj-ukb-cvd/results/projects/22_retinal_risk/data/220602/endpoints.csv
  use_top_n_phecodes: 1000
  filter_input_origins: []
  use_loss_weights: false
  task: tte
  task_kwargs: {}
model:
  model_type: image
  encoder: convnext_small
  freeze_encoder: false
  pretrained: true
  restore_from_ckpt: false
training:
  exclusions_on_losses: false
  exclusions_on_metrics: true
  gradient_checkpointing: false
  contrastive_loss_factor: 0
  endpoint_loss_factor: 1
  vicreg_loss_factor: 0
  node_dropout: null
  normalize_node_embeddings: true
  binarize_records: true
  gradient_checkpoining: true
  write_predictions: true
  write_embeddings: false
  write_attributions: false
  warmup_period: 8
  attribution_baseline_mode: zeros
  use_endpoint_embeddings: false
  patience: 40
  optimizer_kwargs:
    weight_decay: 0.001
    lr: 0.0001
trainer:
  gpus:
  - 0
  precision: 16
  max_epochs: 10000
  num_sanity_val_steps: -1
  detect_anomaly: false
  amp_backend: native
  accumulate_grad_batches: 1
"""

In [10]:
# setup model
from retinalrisk.training import setup_training
import hydra
from omegaconf import DictConfig, OmegaConf

args = OmegaConf.create(yaml)
print(OmegaConf.to_yaml(args))

_, model, _ = setup_training(args)

setup:
  entity: cardiors
  project: retina
  group: null
  name: onnx_export
  root:
    charite-hpc: /sc-projects/sc-proj-ukb-cvd
    eils-hpc: /data/analysis/ag-reils/ag-reils-shared/cardioRS
  data_path: data/2_datasets_pre/211110_anewbeginning/artifacts
  output_path: results/models
  use_data_artifact_if_available: false
  data: null
  restore_id: null
  data_identifier: WandBBaselineData:latest
  tags:
  - baseline_data
head:
  model_type: MLP
  dropout: 0.5
  kwargs:
    num_hidden: 512
    num_layers: 2
    detach_clf: false
    initial_dropout: 0.0
datamodule:
  covariates: []
  augmentation:
    train:
      RandomRotation:
        degrees: 25
      CenterCrop:
        size: 384
      RandomHorizontalFlip:
        p: 0.5
      RandomVerticalFlip:
        p: 0.5
      RandomGrayscale:
        p: 0.1
      RandomPosterize:
        bits: 4
        p: 0.2
      RandomAdjustSharpness:
        sharpness_factor: 2
        p: 0.3
      RandomAutocontrast:
        p: 0.3
    valid:
 

In [15]:
runs_df.best_ckpt.values

array(['/sc-projects/sc-proj-ukb-cvd/results/models/retina/3n2one7r/checkpoints/epoch=112-step=44748.ckpt',
       '/sc-projects/sc-proj-ukb-cvd/results/models/retina/1xcmy2ys/checkpoints/epoch=132-step=39368.ckpt',
       '/sc-projects/sc-proj-ukb-cvd/results/models/retina/37ps9kby/checkpoints/epoch=122-step=37515.ckpt',
       '/sc-projects/sc-proj-ukb-cvd/results/models/retina/lzl9g0gn/checkpoints/epoch=146-step=47334.ckpt',
       '/sc-projects/sc-proj-ukb-cvd/results/models/retina/3uyg9080/checkpoints/epoch=111-step=40992.ckpt',
       '/sc-projects/sc-proj-ukb-cvd/results/models/retina/16f0zmet/checkpoints/epoch=119-step=36000.ckpt'],
      dtype=object)

In [20]:
import torch
import os

output_dir = '/sc-projects/sc-proj-ukb-cvd/results/models/retina/'

# Input to the model
x = torch.randn(*IMG_SHAPE, requires_grad=True)

for i in range(len(runs_df)):
    run = runs_df.loc[i]
    ckpt_path = run['best_ckpt']
    run_id = run['id']
    out_path = os.path.join(output_dir, run_id, 'onnx')
    os.makedirs(out_path, exist_ok=True)
    
    if torch.cuda.is_available():
            device = torch.device('cuda')
    else:
        device = torch.device('cpu')

    ckpt = torch.load(ckpt_path, map_location=device)
    model.load_state_dict(ckpt["state_dict"])
    model.eval()

    torch_out = model(x)
    # Export the model
    torch.onnx.export(model,                     # model being run
                      x,                         # model input (or a tuple for multiple inputs)
                      os.path.join(out_path, 'best_checkpoint.onnx'),   # where to save the model (can be a file or file-like object)
                      export_params=True,        # store the trained parameter weights inside the model file
                      opset_version=10,          # the ONNX version to export the model to
                      do_constant_folding=True,  # whether to execute constant folding for optimization
                      input_names = ['input'],   # the model's input names
                      output_names = ['output'], # the model's output names
                      dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                    'output' : {0 : 'batch_size'}})

AttributeError: 'Tensor' object has no attribute 'covariates'

In [12]:
#model

torch_model.eval()
torch_out = model(x)

# Export the model
torch.onnx.export(torch_model,               # model being run
                  x,                         # model input (or a tuple for multiple inputs)
                  "super_resolution.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})